#Silver Data Transformations

In [0]:
df=spark.read.format('delta').option('header',True).option('inferschema',True)\
.load("abfss://bronze@anamikanetflixdl.dfs.core.windows.net/netflix_titles")
df.display()

In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})
df.display()